In [8]:
import pandas as pd
import json
import numpy as np

# チームIDをチーム名に変換する辞書作成
team_df = pd.read_csv('/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/team.csv')
team_dic = dict(zip(team_df['team_id'], team_df['team_name']))
team_id_list = [v for v in team_dic.keys()]


# ○: フル出場, ▲: 途中出場, ▽: 途中退場, ◆: 途中出場途中退場, ×: 出場停止, ※: 他大会の影響で出場停止, B: ベンチ入り ×××:ベンチ外
# 各々の回数をカウント
characters = ['○', '▲', '▽', '◆', 'B', '×', "-"]


#記号を出場時間に変換する関数を定義
def convert_sign(self):
    import re
    
    if "○" in self:
        return int(90)
    
    elif "▽" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)

    elif "▲" in self:        
        num = re.sub(r"\D", "", self)
        return int(num)    
    
    elif "◆" in self:        
            num = re.sub(r"\D", "", self)
            return int(num)
    
    else:
        return int(0)

pretime_2018 = pd.DataFrame({"name":[], "pretime":[]})
pretime_2017 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2016 = pd.DataFrame({"name":[], "pretime":[]})    
pretime_2015 = pd.DataFrame({"name":[], "pretime":[]})    

df_dic = {2018:pretime_2018, 2017:pretime_2017, 2016:pretime_2016, 2015:pretime_2015}
for year in [2018,2017,2016,2015]:
    # 2015シーズンのjsonをcsvに変換
    #year = 2015
    team_df_year = team_df[team_df.year == year]
    #df = pd.DataFrame({"name":[], "pretime":[]})
    df = pd.DataFrame()
    for div, team_id in zip(team_df_year["div"], team_df_year["team_id"]):
        # 2stageかどうか
        is_2stage = year < 2017 and div == "J1"

        # 後で１試合あたりの指標に変換するために試合数の情報を記憶
        if div == 'J1':
            num_match = 34
        else:
            num_match = 42

        if is_2stage:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s1.json", encoding="utf-8_sig") as f:
                first_dic = json.load(f)
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}_s2.json", encoding="utf-8_sig") as f:
                second_dic = json.load(f)

            first_df = pd.DataFrame.from_dict(first_dic, orient='index')
            second_df = pd.DataFrame.from_dict(second_dic, columns=list(range(17, 34)), orient='index')
            #(first_df)
            full_df = pd.concat([first_df, second_df], axis=1, join='outer')

        else:
            with open(f"/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/input/event/event_play_{year}_{div}_{team_id}.json", encoding="utf-8_sig") as f:
                dic = json.load(f)

            full_df = pd.DataFrame.from_dict(dic, orient='index')

        #４シーズンに分けるための数値作成
        num_to_div_list = [(num_match//8)*n for n in range(1,9)]
        st1 = num_to_div_list[0]
        st2 = num_to_div_list[1]
        st3 = num_to_div_list[2]
        st4 = num_to_div_list[3]
        st5 = num_to_div_list[4]
        st6 = num_to_div_list[5]
        st7 = num_to_div_list[6]
        st8 = num_match
        
        #関数を適用し、かつ4シーズンに分ける
        full_df = full_df.replace('', np.nan, regex=True)
        full_df = full_df.fillna("-")

        #記号を出場時間に変換
        full_df_con = full_df.iloc[:, :num_match].applymap(convert_sign)
        full_df_con["pretime"] = full_df_con.sum(axis=1)
        full_df_con["1st_pretime"] = full_df_con.iloc[:, :st1].sum(axis=1)
        full_df_con["2st_pretime"] = full_df_con.iloc[:, st1:st2].sum(axis=1)
        full_df_con["3st_pretime"] = full_df_con.iloc[:, st2:st3].sum(axis=1)
        full_df_con["4st_pretime"] = full_df_con.iloc[:, st3:st4].sum(axis=1)
        full_df_con["5st_pretime"] = full_df_con.iloc[:, st4:st5].sum(axis=1)
        full_df_con["6st_pretime"] = full_df_con.iloc[:, st5:st6].sum(axis=1)
        full_df_con["7st_pretime"] = full_df_con.iloc[:, st6:st7].sum(axis=1)
        full_df_con["8st_pretime"] = full_df_con.iloc[:, st7:num_match].sum(axis=1)
        
        for ch in characters:
            full_df_con["total_" + ch] = full_df.apply(lambda d: d.str.contains(ch)).sum(axis=1).values / num_match
            full_df_con["1st_" + ch] = full_df.iloc[:, :st1].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["2st_" + ch] = full_df.iloc[:, st1:st2].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["3st_" + ch] = full_df.iloc[:, st2:st3].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["4st_" + ch] = full_df.iloc[:, st3:st4].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["5st_" + ch] = full_df.iloc[:, st4:st5].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["6st_" + ch] = full_df.iloc[:, st5:st6].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["7st_" + ch] = full_df.iloc[:, st6:st7].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8)
            full_df_con["8st_" + ch] = full_df.iloc[:, st7:num_match].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / (num_match//8 + 2)

        new_df = pd.DataFrame()
        new_df["name"] = full_df_con.index
        for i in ['pretime', '1st_pretime', '2st_pretime',
       '3st_pretime', '4st_pretime', '5st_pretime', '6st_pretime',
       '7st_pretime', '8st_pretime',     'total_○',       '1st_○',
             '2st_○',       '3st_○',       '4st_○',       '5st_○',
             '6st_○',       '7st_○',       '8st_○',     'total_▲',
             '1st_▲',       '2st_▲',       '3st_▲',       '4st_▲',
             '5st_▲',       '6st_▲',       '7st_▲',       '8st_▲',
           'total_▽',       '1st_▽',       '2st_▽',       '3st_▽',
             '4st_▽',       '5st_▽',       '6st_▽',       '7st_▽',
             '8st_▽',     'total_◆',       '1st_◆',       '2st_◆',
             '3st_◆',       '4st_◆',       '5st_◆',       '6st_◆',
             '7st_◆',       '8st_◆',     'total_B',       '1st_B',
             '2st_B',       '3st_B',       '4st_B',       '5st_B',
             '6st_B',       '7st_B',       '8st_B',     'total_×',
             '1st_×',       '2st_×',       '3st_×',       '4st_×',
             '5st_×',       '6st_×',       '7st_×',       '8st_×',
           'total_-',       '1st_-',       '2st_-',       '3st_-',
             '4st_-',       '5st_-',       '6st_-',       '7st_-',
             '8st_-']:
            new_df[i] = full_df_con[i].values
        new_df['team_id'] = team_id
        new_df['team'] = new_df['team_id'].map(int).map(team_dic)
        
        df = pd.concat([df, new_df])

 
    df["name-team"] = df["name"] + "-" + df["team"]
    df.drop(["name"], axis=1, inplace=True)
    df.drop(["team"], axis=1, inplace=True)


    #出場時間を出場割合に変換
    df["time_rate"] = df["pretime"].copy()
    df["1st_time_rate"] = df["1st_pretime"].copy()
    df["2st_time_rate"] = df["2st_pretime"].copy()
    df["3st_time_rate"] = df["3st_pretime"].copy()
    df["4st_time_rate"] = df["4st_pretime"].copy()
    df["5st_time_rate"] = df["5st_pretime"].copy()
    df["6st_time_rate"] = df["6st_pretime"].copy()
    df["7st_time_rate"] = df["7st_pretime"].copy()
    df["8st_time_rate"] = df["8st_pretime"].copy()
    for i in team_id_list:
        s = df.loc[df["team_id"]==i, "pretime"].sum()
        df.loc[df["team_id"]==i, "time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "1st_pretime"].sum()
        df.loc[df["team_id"]==i, "1st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "2st_pretime"].sum()
        df.loc[df["team_id"]==i, "2st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "3st_pretime"].sum()
        df.loc[df["team_id"]==i, "3st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "4st_pretime"].sum()
        df.loc[df["team_id"]==i, "4st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "5st_pretime"].sum()
        df.loc[df["team_id"]==i, "5st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "6st_pretime"].sum()
        df.loc[df["team_id"]==i, "6st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "7st_pretime"].sum()
        df.loc[df["team_id"]==i, "7st_time_rate"] /= (s/100)
        s = df.loc[df["team_id"]==i, "8st_pretime"].sum()
        df.loc[df["team_id"]==i, "8st_time_rate"] /= (s/100)
        
    for i in ["pretime", "1st_pretime", "2st_pretime", "3st_pretime", "4st_pretime",
             "5st_pretime", "6st_pretime", "7st_pretime", "8st_pretime"]:
        df.drop([i], axis=1, inplace=True)

    df.drop(["team_id"], axis=1, inplace=True)
    df_dic[year] = df
    
pretime_2018 = df_dic[2018]
pretime_2017 = df_dic[2017]
pretime_2016 = df_dic[2016]
pretime_2015 = df_dic[2015]

In [9]:
pretime_2018.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/8div_pretime_2018_記号カウント.csv", index=False)
pretime_2017.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/8div_pretime_2017_記号カウント.csv", index=False)
pretime_2016.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/8div_pretime_2016_記号カウント.csv", index=False)
pretime_2015.to_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/8div_pretime_2015_記号カウント.csv", index=False)


In [4]:
a = pd.read_csv("/Users/onehe/Desktop/プログラミング/コンペティション/Jリーグ選手出場時間予測_nishika/data/output/pretime_2018.csv")
a.head(3)

,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,クォン スンテ-鹿島,7.222041,9.090909,6.829392,4.545455,8.181818
1,内田 篤人-鹿島,1.789164,2.032828,4.122929,1.123737,0.262626
2,昌子 源-鹿島,4.095462,6.818182,5.526748,0.517677,3.636364


In [94]:
pretime_2016.head(30)

,total_○,1st_○,2st_○,3st_○,4st_○,total_▲,1st_▲,2st_▲,3st_▲,4st_▲,total_▽,1st_▽,2st_▽,3st_▽,4st_▽,total_◆,1st_◆,2st_◆,3st_◆,4st_◆,total_B,1st_B,2st_B,3st_B,4st_B,total_×,1st_×,2st_×,3st_×,4st_×,total_-,1st_-,2st_-,3st_-,4st_-,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,1.000000,1.000,1.000,1.000,1.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,林 卓人-広島,9.139785,9.090909,9.090909,9.090909,9.259259
1,0.441176,0.500,0.625,0.000,0.6,0.000000,0.000,0.000,0.000,0.0,0.058824,0.000,0.125,0.000,0.1,0.000000,0.000,0.0,0.0,0.0,0.117647,0.500,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.382353,0.000,0.250,1.000,0.3,水本 裕貴-広島,4.501195,4.545455,6.578283,0.000000,6.440329
2,0.823529,1.000,0.875,1.000,0.5,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.029412,0.0,0.125,0.000,0.0,0.147059,0.000,0.000,0.000,0.5,千葉 和彦-広島,7.526882,9.090909,7.954545,9.090909,4.629630
3,0.705882,0.875,0.750,0.125,1.0,0.088235,0.000,0.125,0.250,0.0,0.029412,0.000,0.125,0.000,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.176471,0.125,0.000,0.625,0.0,青山 敏弘-広島,6.765233,7.954545,7.601010,1.679293,9.259259
4,0.441176,0.250,0.875,0.375,0.3,0.029412,0.000,0.000,0.000,0.1,0.235294,0.125,0.125,0.625,0.1,0.000000,0.000,0.0,0.0,0.0,0.205882,0.250,0.000,0.000,0.5,0.000000,0.0,0.000,0.000,0.0,0.088235,0.375,0.000,0.000,0.0,森﨑 和幸-広島,5.737754,3.371212,8.724747,7.916667,3.456790
5,0.382353,0.500,0.625,0.375,0.1,0.088235,0.125,0.000,0.000,0.2,0.500000,0.375,0.375,0.625,0.6,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.029412,0.000,0.000,0.000,0.1,ピーター ウタカ-広島,7.855436,7.979798,8.939394,8.737374,6.152263
6,0.147059,0.000,0.125,0.500,0.0,0.205882,0.375,0.500,0.000,0.0,0.058824,0.125,0.000,0.125,0.0,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.588235,0.500,0.375,0.375,1.0,浅野 拓磨-広島,2.377539,1.628788,2.790404,5.631313,0.000000
7,0.000000,0.000,0.000,0.000,0.0,0.323529,0.125,0.375,0.250,0.5,0.235294,0.375,0.000,0.375,0.2,0.000000,0.000,0.0,0.0,0.0,0.382353,0.500,0.375,0.375,0.3,0.000000,0.0,0.000,0.000,0.0,0.058824,0.000,0.250,0.000,0.0,佐藤 寿人-広島,1.986260,2.424242,0.719697,2.967172,1.862140
8,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.000,0.0,0.0,0.0,1.000000,1.000,1.000,1.000,1.0,0.000000,0.0,0.000,0.000,0.0,0.000000,0.000,0.000,0.000,0.0,増田 卓也-広島,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.352941,0.375,0.375,0.250,0.4,0.088235,0.000,0.000,0.375,0.0,0.352941,0.625,0.250,0.125,0.4,0.000000,0.000,0.0,0.0,0.0,0.000000,0.000,0.000,0.000,0.0,0.000000,0.0,0.000,0.000,0.0,0.205882,0.000,0.375,0.250,0.2,ミキッチ-広島,6.072282,8.181818,5.151515,4.166667,6.656379


In [73]:
pd.set_option("display.max_columns", 100)
full_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,○,○,○,○,○,○,○,○,▽45,,,,B,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○
田向 泰輝,,▽56,B,B,○,B,B,B,B,○,B,○,B,B,B,B,B,○,▽87,○,○,▽89,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,▽77
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,▽89,▽74,○,○,○,○,○,○,○,○,○,○,○,○,○,○


In [85]:
a = full_df.fillna('""')
full_df.iloc[0,12]

''

In [89]:
import numpy as np
full_df = full_df.replace('', np.nan, regex=True)
full_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41
本間 幸司,B,B,B,○,○,○,○,○,○,○,○,▽45,NaN,NaN,NaN,B,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○
田向 泰輝,NaN,▽56,B,B,○,B,B,B,B,○,B,○,B,B,B,B,B,○,▽87,○,○,▽89,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,▽77
田中 雄大,○,○,○,○,○,○,▽89,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,○,▽89,▽74,○,○,○,○,○,○,○,○,○,○,○,○,○,○


In [92]:
new_df

,name,pretime,1st_pretime,2st_pretime,3st_pretime,4st_pretime,total_○,1st_○,2st_○,3st_○,4st_○,total_▲,1st_▲,2st_▲,3st_▲,4st_▲,total_▽,1st_▽,2st_▽,3st_▽,4st_▽,total_◆,1st_◆,2st_◆,3st_◆,4st_◆,total_B,1st_B,2st_B,3st_B,4st_B,total_×,1st_×,2st_×,3st_×,4st_×,total_-,1st_-,2st_-,3st_-,4st_-,team_id,team
0,本間 幸司,3105,630,495,900,1080,34,7,5,10,12,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,4,3,1,0,0,0,0,0,0,0,3,0,3,0,0,94,水戸
1,田向 泰輝,2559,236,357,899,1067,25,2,3,9,11,0,0,0,0,0,4,1,1,1,1,0,0,0,0,0,12,6,6,0,0,0,0,0,0,0,1,1,0,0,0,94,水戸
2,田中 雄大,3762,899,900,883,1080,39,9,10,8,12,0,0,0,0,0,3,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,94,水戸
3,山﨑 貴雅,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,6,4,2,0,0,0,0,0,0,0,35,5,8,10,12,94,水戸
4,金 聖基,1450,720,634,96,0,16,8,7,1,0,6,0,2,4,0,0,0,0,0,0,0,0,0,0,0,13,2,0,5,6,0,0,0,0,0,7,0,1,0,6,94,水戸
5,石神 幸征,1470,621,231,2,616,12,5,1,0,6,5,1,1,2,1,3,1,1,0,1,1,1,0,0,0,9,1,3,4,1,1,1,0,0,0,11,0,4,4,3,94,水戸
6,鈴木 雄斗,1789,384,291,88,1026,10,1,1,0,8,6,1,4,1,0,11,4,2,1,4,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,11,0,3,8,0,94,水戸
7,岩尾 憲,3690,900,900,900,990,41,10,10,10,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,94,水戸
8,小谷野 顕治,373,92,270,0,11,1,0,1,0,0,7,2,4,0,1,3,1,2,0,0,0,0,0,0,0,24,7,2,9,6,0,0,0,0,0,7,0,1,1,5,94,水戸
9,船谷 圭祐,2375,88,874,568,845,9,0,7,0,2,7,1,0,4,2,18,1,3,6,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,8,0,0,0,94,水戸


In [55]:
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,total_○,1st_○,2st_○,3st_○,4st_○,total_▲,1st_▲,2st_▲,3st_▲,4st_▲,total_▽,1st_▽,2st_▽,3st_▽,4st_▽,total_◆,1st_◆,2st_◆,3st_◆,4st_◆,total_B,1st_B,2st_B,3st_B,4st_B,total_×,1st_×,2st_×,3st_×,4st_×,34,35,36,37,38,39,40,41,name-team,time_rate,1st_time_rate,2st_time_rate,3st_time_rate,4st_time_rate
0,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,34,8,8,8,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,林 卓人-広島,9.090909,9.090909,9.090909,9.090909,9.090909
1,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,76,90,90,90,90,90,90,90,90,90,67,0,31,8,8,7,8,0,0,0,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,水本 裕貴-広島,8.713607,9.090909,9.090909,8.914141,7.949495
2,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,90,34,8,8,8,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,千葉 和彦-広島,9.090909,9.090909,9.090909,9.090909,9.090909


In [46]:
a = pd.DataFrame()
a

""


In [14]:
stage = ["total_", "1st_", "2st_", "3st_", "4st_"]
characters = ['○', '▲', '▽', '◆', 'B', '×']
dt = pd.DataFrame()
for st in stage:
    for ch in characters:
        dt[st + ch] = 1

dt

,total_○,total_▲,total_▽,total_◆,total_B,total_×,1st_○,1st_▲,1st_▽,1st_◆,...,3st_▽,3st_◆,3st_B,3st_×,4st_○,4st_▲,4st_▽,4st_◆,4st_B,4st_×


In [ ]:
        for ch in range(0, 6):
            for i in range(0, 5):
                if i == 0:
                    div_num = num_match
                elif i == 4:
                    div_num = num_match//4 + 2
                else:
                    div_num = num_match//4
                full_df[stage[i] + characters[ch]] = full_df.iloc[:, :column[i]].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / div_num

In [ ]:
        ''''
        #記号をカウント
        stage = ["total_", "1st_", "2st_", "3st_", "4st_"]
        column = [num_match, first, second, third, forth]
        for ch in characters:
            for st, col in zip(stage, column):
                if st == "total_":
                    div_num = num_match
                elif st == "4st":
                    div_num = num_match//4 + 2
                else:
                    div_num = num_match//4
                full_df[st + ch] = full_df.iloc[:, :col].apply(lambda d: d.str.contains(ch)).sum(axis=1).values / div_num
        ''''

In [63]:
new_df.columns

Index([       'name',             0,             1,             2,
                   3,             4,             5,             6,
                   7,             8,             9,            10,
                  11,            12,            13,            14,
                  15,            16,            17,            18,
                  19,            20,            21,            22,
                  23,            24,            25,            26,
                  27,            28,            29,            30,
                  31,            32,            33,            34,
                  35,            36,            37,            38,
                  39,            40,            41,     'pretime',
       '1st_pretime', '2st_pretime', '3st_pretime', '4st_pretime',
           'total_○',       '1st_○',       '2st_○',       '3st_○',
             '4st_○',     'total_▲',       '1st_▲',       '2st_▲',
             '3st_▲',       '4st_▲',     'total_▽',       '1st

In [66]:
full_df_con.columns

Index([            0,             1,             2,             3,
                   4,             5,             6,             7,
                   8,             9,            10,            11,
                  12,            13,            14,            15,
                  16,            17,            18,            19,
                  20,            21,            22,            23,
                  24,            25,            26,            27,
                  28,            29,            30,            31,
                  32,            33,     'pretime', '1st_pretime',
       '2st_pretime', '3st_pretime', '4st_pretime',     'total_○',
             '1st_○',       '2st_○',       '3st_○',       '4st_○',
           'total_▲',       '1st_▲',       '2st_▲',       '3st_▲',
             '4st_▲',     'total_▽',       '1st_▽',       '2st_▽',
             '3st_▽',       '4st_▽',     'total_◆',       '1st_◆',
             '2st_◆',       '3st_◆',       '4st_◆',     'total

In [7]:
print(full_df_con.columns[41:])

Index([           41,     'pretime', '1st_pretime', '2st_pretime',
       '3st_pretime', '4st_pretime', '5st_pretime', '6st_pretime',
       '7st_pretime', '8st_pretime',     'total_○',       '1st_○',
             '2st_○',       '3st_○',       '4st_○',       '5st_○',
             '6st_○',       '7st_○',       '8st_○',     'total_▲',
             '1st_▲',       '2st_▲',       '3st_▲',       '4st_▲',
             '5st_▲',       '6st_▲',       '7st_▲',       '8st_▲',
           'total_▽',       '1st_▽',       '2st_▽',       '3st_▽',
             '4st_▽',       '5st_▽',       '6st_▽',       '7st_▽',
             '8st_▽',     'total_◆',       '1st_◆',       '2st_◆',
             '3st_◆',       '4st_◆',       '5st_◆',       '6st_◆',
             '7st_◆',       '8st_◆',     'total_B',       '1st_B',
             '2st_B',       '3st_B',       '4st_B',       '5st_B',
             '6st_B',       '7st_B',       '8st_B',     'total_×',
             '1st_×',       '2st_×',       '3st_×',       '4st